In [1]:
import pandas as pd
import numpy as np
import censusdata
import json
import math
import pickle
import os
from datetime import datetime
import time

Functions to save and load `.pkl` files

In [2]:
def save_pickle(obj, filepath):
    """
    Save an object as a .pkl file, with specified filename (no need to add .pkl suffix).
    The filename must be a string.
    Filepath is already specified as: 'data/' + filename + '.pkl'
    """
    with open(filepath, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        print('Saved', filepath)

In [3]:
def load_pickle(filepath):
    """
    Load a .pkl file from the `data/` folder.
    The name must be a string.
    Filepath is already specified as: 'data/' + name + '.pkl'
    """
    with open(filepath, 'rb') as f:
        return pickle.load(f)

Read in Census API key

In [4]:
with open('../../conf/census_api_secret.json', 'rb') as f:
    data = f.read()
    census_api_secret = json.loads(data)
api_key = census_api_secret['census_api_key']

#### Build `states_df`, containing state-level identifying info

In [5]:
states_dict = censusdata.geographies(censusdata.censusgeo([('state', '*')]),
                                       'acs1', 2017, key=api_key)
len(states_dict)

52

In [6]:
names = list(states_dict.keys())
ids = [x.geo[0][1] for x in list(states_dict.values())]
state_df = dict(zip(names, ids))
state_df = pd.DataFrame({'state_name': list(state_df.keys()),
                         'census_id': list(state_df.values())
                        })

In [7]:
with open('../../data/01_raw/states.json', 'rb') as f:
    data = f.read()
    states_json = json.loads(data)

In [8]:
states_json_df = pd.DataFrame({'state_name': list(states_json.keys()),
                               'state_abbr': list(states_json.values())
                              }) 
state_df = pd.merge(state_df, states_json_df, how='left', on='state_name')
print(state_df.shape)
state_df.head()

(52, 3)


,state_name,census_id,state_abbr
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA


Build `districts_dict` for 1 year (2017)

In [9]:
districts_dict = censusdata.geographies(censusdata.censusgeo([('congressional district', '*')]),
                                       'acs1', 2017, key=api_key)
len(districts_dict)

437

In [10]:
list(districts_dict.keys())[:5]

['Congressional District 1 (115th Congress), Alabama',
 'Congressional District 2 (115th Congress), Alabama',
 'Congressional District 3 (115th Congress), Alabama',
 'Congressional District 4 (115th Congress), Alabama',
 'Congressional District 5 (115th Congress), Alabama']

In [11]:
list(districts_dict.values())[:5]

[censusgeo((('state', '01'), ('congressional district', '01'))),
 censusgeo((('state', '01'), ('congressional district', '02'))),
 censusgeo((('state', '01'), ('congressional district', '03'))),
 censusgeo((('state', '01'), ('congressional district', '04'))),
 censusgeo((('state', '01'), ('congressional district', '05')))]

In [12]:
al_01 = list(districts_dict.values())[0]
al_01

censusgeo((('state', '01'), ('congressional district', '01')))

In [13]:
censusdata.printtable(censusdata.censustable('acs1', 2017, 'B23025'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B23025_001E  | EMPLOYMENT STATUS FOR THE POPU | !! Estimate Total                                        | int  
B23025_002E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total In labor force                      | int  
B23025_003E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Civilian labor fo | int  
B23025_004E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_005E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! !! Estimate Total In labor force Civilian labor | int  
B23025_006E  | EMPLOYMENT STATUS FOR THE POPU | !! !! !! Estimate Total In labor force Armed Forces      | int  
B23025_007E  | EMPLOYMENT STATUS FOR THE POPU | !! !! Estimate Total Not in labor force      

#### Read in CSV of our Census variables of interest

First go through cleaning process in `Sublime`:
- find/replace all: "\| int" with ""
- find/replace all: "\| float" with ""
- find/replace all: "!! +" with ""
- find/replace all: "estimate total " with ""
- fnid/replace all: "estimate "
- copy/paste into excel; text-to-columns with pipe delimiter
- add header row: variable, table, label
- save as csv

In [14]:
varnames = pd.read_csv('../../data/01_raw/census_variable_list_20190809.csv')
print(varnames.shape)
varnames.tail()

(190, 3)


,variable,table,label
185,B15002_018E,SEX BY EDUCATIONAL ATTAINMENT,Male Doctorate degree
186,B15002_032E,SEX BY EDUCATIONAL ATTAINMENT,Female Bachelor's degree
187,B15002_033E,SEX BY EDUCATIONAL ATTAINMENT,Female Master's degree
188,B15002_034E,SEX BY EDUCATIONAL ATTAINMENT,Female Professional school degree
189,B15002_035E,SEX BY EDUCATIONAL ATTAINMENT,Female Doctorate degree


In [15]:
# varnames = pd.read_csv('../../data/01_raw/census_variable_table_crosswalk.csv')
# print(varnames.shape)
# varnames.head()

In [16]:
varnames['varname_long'] = (varnames.table.str.strip() + '_' 
                + varnames.label.str.strip()).str.replace(' ', '_')
varnames['variable'] = varnames['variable'].str.strip()

In [17]:
varnames.head()

,variable,table,label,varname_long
0,B01003_001E,TOTAL POPULATION,,TOTAL_POPULATION_
1,B01001_002E,SEX BY AGE,Male,SEX_BY_AGE_Male
2,B01001_026E,SEX BY AGE,Female,SEX_BY_AGE_Female
3,B01001_004E,SEX BY AGE,Male 5 to 9 years,SEX_BY_AGE_Male_5_to_9_years
4,B01001_005E,SEX BY AGE,Male 10 to 14 years,SEX_BY_AGE_Male_10_to_14_years


#### Loop over `year` and `district`, passing in the same `varlist` each time

Do you need to first make a call to `censusdata.geographies()` to get districts for each year?

In [18]:
# varlist stays the same throughout
varlist = list(varnames.variable.str.strip().values)#[:5]
len(varlist)

190

In [19]:
new_cols = list(varnames.varname_long.str.strip().values)#[:5]
len(new_cols)

190

#### Loop over years

#### Loop over all states, and districts within each state 

In [20]:
start = datetime.now()
years = [2017, 2016]  # change to all years
census_data = pd.DataFrame()
new_idxs = []
num_of_splits = math.ceil(len(varlist)/25)  # finalize varlist
varlist_split = np.array_split(varlist, num_of_splits)

for year in years:
    print('Year:', year)
    
    year_data = pd.DataFrame()
    for state_abbr in state_df.state_abbr.values[:1]:  # change to all states
        census_id = state_df[state_df.state_abbr == state_abbr]['census_id'].values[0]
        print('state census ID:', census_id)

        districts = censusdata.geographies(censusdata.censusgeo([('state', f'{census_id}'), 
                                                                      ('congressional district', '*')]),
                                          'acs1', year, key=api_key
                                           )
        for district in list(districts.values()):
            print(district)
            state_id = district.geo[0][1]
            district_id = district.geo[1][1]
            state_abbr = state_df[state_df.census_id == state_id]['state_abbr'].values[0]
            colname = state_abbr + '-' + district_id
            print(colname)
            new_idxs.append(colname)

            new_row = pd.DataFrame()
            for varlist in varlist_split:
                new_row = pd.concat([new_row, censusdata.download('acs1', year, district, list(varlist), key=api_key)],
                                    axis=1)
                time.sleep(3)

            year_data = pd.concat([year_data, new_row], sort='False')

    year_data['year'] = year
    census_data = pd.concat([census_data, year_data], sort='False')
    
census_data.index = new_idxs
end = datetime.now()
runtime = end - start
print('Runtime was', runtime.seconds/60, 'minutes')

Year: 2017
state census ID: 01
Summary level: 500, state:01> congressional district:01
AL-01
Summary level: 500, state:01> congressional district:02
AL-02
Summary level: 500, state:01> congressional district:03
AL-03
Summary level: 500, state:01> congressional district:04
AL-04
Summary level: 500, state:01> congressional district:05
AL-05
Summary level: 500, state:01> congressional district:06
AL-06
Summary level: 500, state:01> congressional district:07
AL-07
Year: 2016
state census ID: 01
Summary level: 500, state:01> congressional district:01
AL-01


ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [133]:
census_data.shape

(0, 0)

In [115]:
census_data#.head()

,B01001_002E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,...,B25075_002E,B25075_003E,B25075_004E,B25075_005E,B25075_006E,B25075_007E,B25075_008E,B25075_009E,B25075_010E,year
AL-01,343769,21687,24958,15014,7992,4696,3634,12679,23832,21246,...,2613,3317,1596,2469,2353,2033,1424,3204,5304,2017
AL-02,328474,21345,23661,13221,10058,5226,2959,13831,24024,20319,...,5015,2733,1828,2770,2001,2872,1539,4140,5902,2017
AL-03,346742,22945,24502,12872,11286,7198,5424,15010,22041,20287,...,7071,2719,2649,3658,3833,4537,2947,5592,8453,2017
AL-04,332923,23278,21585,12877,10731,4174,3570,11811,21088,20022,...,5547,3992,2733,3569,3035,4317,3047,6954,7726,2017
AL-05,353533,22442,23188,14620,9218,3202,6043,14823,25075,23413,...,3670,1473,1854,1012,1930,1330,1054,4152,5357,2017
AL-06,333957,21334,23391,14584,8743,3316,3897,11952,19905,21511,...,1991,1756,1176,1267,2064,1971,1040,3203,4043,2017
AL-07,320498,20911,21470,12724,13906,4829,5912,18538,25146,21668,...,4311,3493,2155,2604,2785,3931,2912,5996,9371,2017
AL-01,338759,22527,23756,15335,8611,5007,4332,12037,22600,20009,...,3257,1460,1616,1175,1711,3121,2773,5313,6425,2016
AL-02,332477,24514,20603,14324,8992,4363,4002,14589,24093,22975,...,4698,2888,2683,2382,2460,3612,2130,5873,7329,2016
AL-03,344900,25641,20409,15045,10236,5844,6586,15947,24515,20346,...,6895,3618,3057,3045,2890,2569,2000,6789,7511,2016


In [298]:
# save_pickle(census_data, '../../data/01_raw/census_data_test_first5states.pkl')

Saved ../../data/01_raw/census_data_test_first5states.pkl


In [301]:
first5 = load_pickle('../../data/01_raw/census_data_test_first5states.pkl')
print(first5.shape)
first5

(74, 38)


,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,...,B01001_041E,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,year
AL-01,21246,19142,20959,21645,23300,23544,8251,14562,8853,11012,...,25406,11723,15981,10022,13766,16407,12217,8733,7618,2017
AL-02,20319,21825,19832,20134,21704,23200,6943,11554,6758,9615,...,25923,8795,12111,6388,10195,16997,11983,7655,7880,2017
AL-03,20287,21852,21769,23661,20710,22428,10108,11776,7240,9416,...,24660,9204,14102,8180,13086,17768,9594,9153,7696,2017
AL-04,20022,18071,22832,21032,22823,22972,9317,12714,6753,11372,...,23976,9310,14790,8222,13930,17406,12479,9601,8522,2017
AL-05,23413,22923,20894,22744,25046,24962,10271,12426,6951,11201,...,26327,9823,13885,8336,11252,17628,11635,9766,6936,2017
AL-06,21511,25475,21105,24496,21627,22136,8799,12745,7666,9763,...,23545,9293,15506,8587,11714,14590,11575,8169,8533,2017
AL-07,21668,18230,16705,17972,19261,19320,8472,12551,8739,8677,...,21413,10828,15525,9352,11672,13243,10478,7927,7478,2017
AK-00,29715,28346,21871,22454,24117,25189,10980,12589,7785,9857,...,23814,9336,12287,6495,10367,10438,6322,4343,3537,2017
AZ-01,23920,23919,22421,21244,19460,23679,8289,14751,9824,13014,...,25808,9970,15312,9699,14919,17872,14070,8196,7364,2017
AZ-02,22721,20963,18102,19982,20941,23378,7620,13365,9248,12814,...,21517,11923,17344,9467,16411,21110,17738,8874,11018,2017


Display the table in alternate format

In [ ]:
census_data.drop('variable', axis=1).set_index('varname_long').T